# 非线性规划模型
求解非线性规划可以使用scipy.optimize模块、cvxopt库、cvxpy库

## 1.用scipy.optimize模块的minimize函数求解

无约束非线性规划

In [4]:
from scipy.optimize import minimize
import numpy as np

def obj(x):
    x1,x2,x3 = x
    return (2+x1)/(1+x2)-3*x1+4*x3
LB = [0.1]*3; UB = [0.9]*3
bound = tuple(zip(LB,UB))
# print(bound)
res = minimize(obj, np.ones(3),bounds=bound)  #第二个参数为初值
print(res.fun,'\n', res.success,'\n', res.x)  #输出最优值、求解状态、最优解

-0.7736842105263159 
 True 
 [0.9 0.9 0.1]


有约束非线性规划

求解时需要多取几个初始值

In [6]:
from scipy.optimize import minimize
import numpy as np

c1 = np.array([1,1,3,4,2]); c2 = np.array([-8,-2,-3,-1,-2])
A = np.array([[1,1,1,1,1],[1,2,2,1,6],[2,1,6,0,0],[0,0,1,1,5]])
b = np.array([400,800,200,200])
obj = lambda x: np.dot(-c1, x**2)+np.dot(-c2, x)
cons = {'type':'ineq', 'fun': lambda x: b-A@x}  #不等式约束 g(x) >= 0
bd = [(0,99) for i in range(A.shape[1])]  #变量边界（如 [(0, None), (0, 1)]）
# print(bd)
res = minimize(obj, np.ones(5)*90, constraints=cons, bounds=bd)
print(res)

     message: Optimization terminated successfully
     success: True
      status: 0
         fun: -51629.929999446875
           x: [ 5.050e+01  9.900e+01  1.107e-10  9.900e+01  2.020e+01]
         nit: 2
         jac: [-9.300e+01 -1.960e+02  3.000e+00 -7.910e+02 -7.880e+01]
        nfev: 13
        njev: 2
 multipliers: [ 0.000e+00  0.000e+00  4.650e+01  1.576e+01]


## 2.用cvxopt.solver模块求解
### 二次规划
若非线性规划的目标函数为决策变量$x$的二次函数，约束条件又全是线性的，就称这种规划为二次规划

cvxopt.solvers模块中二次规划的<span style="color: red">标准型</span>为:

$min$     $\frac{1}{2}x^TPx+q^Tx$

$Ax<=b,$

$Aeq·x=beq$

In [1]:
import numpy as np
from cvxopt import matrix,solvers

n = 3; P = matrix(0.,(n,n))
P[::n+1] = [3,2,1.7]; q = matrix([3,-8.2,-1.95])
A = matrix([[1.0,0,1],[-1,2,0],[0,1,2]]).T
b = matrix([2.,2,3])
Aeq = matrix(1.,(1,n)); beq = matrix(3.)
s = solvers.qp(P,q,A,b,Aeq,beq)
print('最优解为:', s['x'])
print('最优值为:', s['primal objective'])
print(s)

     pcost       dcost       gap    pres   dres
 0: -1.3148e+01 -4.4315e+00  1e+01  1e+00  9e-01
 1: -6.4674e+00 -7.5675e+00  1e+00  3e-16  4e-16
 2: -7.1538e+00 -7.1854e+00  3e-02  1e-16  2e-15
 3: -7.1758e+00 -7.1761e+00  3e-04  2e-16  2e-15
 4: -7.1760e+00 -7.1760e+00  3e-06  1e-16  2e-15
Optimal solution found.
最优解为: [ 8.00e-01]
[ 1.40e+00]
[ 8.00e-01]

最优值为: -7.1759977687772745
{'x': <3x1 matrix, tc='d'>, 'y': <1x1 matrix, tc='d'>, 's': <3x1 matrix, tc='d'>, 'z': <3x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 3.1697573550427715e-06, 'relative gap': 4.4171660264923267e-07, 'primal objective': -7.1759977687772745, 'dual objective': -7.17600093853463, 'primal infeasibility': 1.204205945063742e-16, 'dual infeasibility': 1.790393725068862e-15, 'primal slack': 4.823184668402676e-08, 'dual slack': 2.346333512527366e-06, 'iterations': 4}


## 3.用cvxpy库求解  该环境暂时不能求解

### 求解非线性<span style="color: orange">整数规划问题</span>

In [ ]:
import cvxpy as cp
import numpy as np

c1 = np.array([1,1,3,4,2]); c2 = np.array([-8,-2,-3,-1,-2])
a = np.array([[1,1,1,1,1],[1,2,2,1,6],[2,1,6,0,0],[0,0,1,1,5]])
b = np.array([400,800,200,200])
x = cp.Variable(5, integer=True)
obj = cp.Minimize(cp.sum(cp.multiply(c1, x**2) + cp.multiply(c2, x)))
con = [0<=x,x<=99, a@x<=b]
prob = cp.Problem(obj, con)
# prob.solve(solver=cp.ECOS)
# print('最优值为:', prob.value)
# print('最优解为:', x.value)

# 飞行管理问题

In [4]:
import numpy as np
import pandas as pd

x0 = np.array([150,85,150,145,130,0])
y0 = np.array([140,85,155,50,150,0])
q = np.array([243,236,220.5,159,230,52])
d = np.zeros((6,6)); a0 = np.zeros((6,6)); b0 = np.zeros((6,6))
xy0 = np.c_[x0,y0]
# print(xy0)
for i in range(6):
    for j in range(6):
        d[i,j] = np.linalg.norm(xy0[i]-xy0[j])  #xy0[i] 和 xy0[j] 之间的欧几里得距离||xy0[i]−xy0[j]||_2
# print(d)
d[np.where(d==0)] = np.inf
a0 = np.arcsin(8./d)*180/np.pi
xy1 = x0+1j*y0; xy2 = np.exp(1j*q*np.pi/180)  #1j为虚数
for m in range(6):
    for n in range(6):
        if n!=m:
            b0[m,n] = np.angle((xy2[n]-xy2[m])/(xy1[m]-xy1[n]))
b0 = b0*180/np.pi
with pd.ExcelWriter('fly_data.xlsx') as f:
    pd.DataFrame(a0).to_excel(excel_writer=f, sheet_name="sheet1", index=False)
    pd.DataFrame(b0).to_excel(excel_writer=f, sheet_name="sheet2", index=False)

In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

a0 = pd.read_excel('fly_data.xlsx')      #读入第一个表单
b0 = pd.read_excel('fly_data.xlsx', 1)   #读入第二个表单
a0 = a0.values; b0 = b0.values  #提取数值
# print(a0)
obj = lambda x: np.sum(np.abs(x))
bd = [(-30,30) for i in range (6)]
x0 = np.ones((1,6))
cons = []
for i in range(5):
    for j in range(i+1,6):
        cons.append({"type":'ineq', 'fun': lambda x: np.abs(b0[i,j]+(x[i]+x[j])/2)-a0[i,j]})
res = minimize(obj, np.ones(6), constraints=cons, bounds=bd)
print(res)

     message: Optimization terminated successfully
     success: True
      status: 0
         fun: 0.7909172985328369
           x: [ 2.533e-07  2.533e-07  2.533e-07  2.533e-07  3.955e-01
                3.955e-01]
         nit: 6
         jac: [-1.000e+00 -1.000e+00 -1.000e+00 -1.000e+00  1.000e+00
                1.000e+00]
        nfev: 54
        njev: 6
 multipliers: [ 2.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
